## Imports

In [49]:
import openai
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

### Loading .env file for Open AI api to work
In order to make this work, create a .env file in the root directory with the following value
```
OPENAI_API_KEY=<YOUR API KEY>
```

In [50]:
load_dotenv()

True

### Initialize openai and Qdrant client
- Open ai is used to create embeddings
- Qdrant is used as a vector database

In [51]:
EMBEDDING_MODEL = "text-embedding-3-small"
openai_client = openai.Client()
client = QdrantClient("localhost", port=6333)

### Common Functions

In [52]:
def create_embeddings(products):
    embeddings = []
    for product in products[:2]:
        response = openai_client.embeddings.create(model=EMBEDDING_MODEL, input=product["name"])
        embeddings.append({"id": product["id"], "name": product["name"], "embedding": response.data[0].embedding})
    return embeddings

In [62]:
def search_products(query, threshold=0.8):
    query_embedding = openai.embeddings.create(model=EMBEDDING_MODEL, input=query).data[0].embedding
    search_result = client.search("products", query_vector=query_embedding, limit=3)
    return search_result

### Hypothetical Dataset of products

In [54]:
products = [
    {"id": 1, "name": "Wireless Bluetooth Headphones"},
    {"id": 2, "name": "Smartphone Charger 20W"},
    {"id": 3, "name": "Portable Power Bank 10000mAh"},
    {"id": 4, "name": "Noise Cancelling Earbuds"},
    {"id": 5, "name": "USB-C to Lightning Cable"},
]

### Creating embeddings for the products dataset

In [55]:
product_embeddings = create_embeddings(products)

In [56]:
product_embeddings[0]

{'id': 1,
 'name': 'Wireless Bluetooth Headphones',
 'embedding': [0.020350871607661247,
  -0.03254341334104538,
  -0.023418672382831573,
  -0.03676866367459297,
  -0.04600577801465988,
  -0.08356105536222458,
  0.0025340262800455093,
  0.032071445137262344,
  0.034678515046834946,
  -0.07600954920053482,
  0.001993227982893586,
  0.027216902002692223,
  -0.04429769888520241,
  0.037869926542043686,
  -0.04216260090470314,
  -0.052860572934150696,
  -0.028880031779408455,
  -0.049174714833498,
  -0.0052984184585511684,
  0.024115389212965965,
  -0.007366093806922436,
  0.06346864998340607,
  0.0006946097128093243,
  0.015552515164017677,
  0.044522445648908615,
  0.01790112443268299,
  -0.02905982919037342,
  -0.012057694606482983,
  0.011563249863684177,
  0.028497962281107903,
  -0.06095147505402565,
  -0.01404670812189579,
  0.04600577801465988,
  -0.0001278250420000404,
  -0.022216273471713066,
  -0.03515048325061798,
  0.037869926542043686,
  -0.020598093047738075,
  0.03058811277

## Create collection for products
- Collection is nothing but a set of points/vectors and we can perform search operation within the defined collection.

In [57]:
client.create_collection("products", VectorParams(size=len(product_embeddings[0]["embedding"]), distance=Distance.COSINE))

True

### Creating points/vectors to be stored in the products collection we created

In [58]:
points = [{"id": product["id"], "vector": product["embedding"], "payload": {"name": product["name"]}} for product in product_embeddings]
client.upsert("products", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Performing Similarity Search

In [59]:

    # results = [{"name": result.payload["name"], "score": result.score} for result in search_result if result.score >= threshold]
    # return results if results else "No relevant results found."

In [63]:
query = "Portable Power Bank"
result = search_products(query)
print("Search results:", search_products(query))

Search results: [ScoredPoint(id=2, version=0, score=0.49669033, payload={'name': 'Smartphone Charger 20W'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=0.29755503, payload={'name': 'Wireless Bluetooth Headphones'}, vector=None, shard_key=None, order_value=None)]


In [67]:
for r in result:
    print(r.payload['name'])

Smartphone Charger 20W
Wireless Bluetooth Headphones
